In [ ]:
# Auto-reload frequently changed files
%load_ext autoreload
%autoreload 2
%aimport utils

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
from os.path import join
from web import for_website

from constants import COLUMNS
from utils import (
    read_combined_demographics_df, read_combined_by_country_demographics_df, read_combined_by_site_demographics_df,
    apply_grouped_bar_theme
)

In [ ]:
"""
Common info that should be defined everytime before rendering visualizations
"""
SITES = read_combined_by_site_demographics_df()[COLUMNS.SITE_ID].unique()

# Titles
NUM_SITES = len(SITES)
DATA_DATE = "2020-04-07"
VIS_DATE = "2020-04-09"
NUM_PATIENTS = "15,427"
SUBTITLE = f"Data as of {DATA_DATE} | {NUM_SITES} Sites | Plots generated on {VIS_DATE}"

SAVE_DIR = join("..", "output") # Where to save visualization *.PNG files

# Colors
COMBINED = "All countries"
COMBINED_COLOR = "#444444"

COUNTRIES = ["France", "Germany", "Italy", "Singapore", "USA"]
COUNTRY_COLOR = ["#0072B2", "#E69F00", "#009E73", "#CC79A7", "#D55E00"]
COLOR_BY_COUNTRY = {COUNTRIES[i]: COUNTRY_COLOR[i] for i in range(len(COUNTRIES))} 

COUNTRIES_AND_COMBINED = [COMBINED] + COUNTRIES
COUNTRY_AND_COMBINED_COLOR = [COMBINED_COLOR] + COUNTRY_COLOR
COLOR_BY_COUNTRY_AND_COMBINED = {COUNTRIES_AND_COMBINED[i]: COUNTRY_AND_COMBINED_COLOR[i] for i in range(len(COUNTRIES_AND_COMBINED))} 

COLOR20 = [
    "#3366cc", "#dc3912", "#ff9900", "#109618", "#990099", "#0099c6", 
    "#dd4477", "#66aa00", "#b82e2e", "#316395", "#994499", "#22aa99", 
    "#aaaa11", "#6633cc", "#e67300", "#8b0707", "#651067", "#329262", "#5574a6", "#3b3eac"
]

# Required Setups
- All datasets should be placed in `../data/combined` (e.g., `../data/combined/Demographics-Combinedyymmdd.csv`).

# Data Preprocess

In [ ]:
def preprocess_demo_df(df_dm):
    
    # Drop unused columns before preprocessing for the simplicity
    df_dm = df_dm.drop(columns=[
        COLUMNS.UNMASKED_SITES_TOTAL_PATIENTS,
        COLUMNS.UNMASKED_SITES_AGE_0TO2,
        COLUMNS.UNMASKED_SITES_AGE_3TO5,
        COLUMNS.UNMASKED_SITES_AGE_6TO11,
        COLUMNS.UNMASKED_SITES_AGE_12TO17,
        COLUMNS.UNMASKED_SITES_AGE_18TO25,
        COLUMNS.UNMASKED_SITES_AGE_26TO49,
        COLUMNS.UNMASKED_SITES_AGE_50TO69,
        COLUMNS.UNMASKED_SITES_AGE_70TO79,
        COLUMNS.UNMASKED_SITES_AGE_80PLUS,
        COLUMNS.MASKED_SITES_TOTAL_PATIENTS,
        COLUMNS.MASKED_SITES_AGE_0TO2,
        COLUMNS.MASKED_SITES_AGE_3TO5,
        COLUMNS.MASKED_SITES_AGE_6TO11,
        COLUMNS.MASKED_SITES_AGE_12TO17,
        COLUMNS.MASKED_SITES_AGE_18TO25,
        COLUMNS.MASKED_SITES_AGE_26TO49,
        COLUMNS.MASKED_SITES_AGE_50TO69,
        COLUMNS.MASKED_SITES_AGE_70TO79,
        COLUMNS.MASKED_SITES_AGE_80PLUS,
        COLUMNS.MASKED_UPPER_BOUND_TOTAL_PATIENTS,
        COLUMNS.TOTAL_PATIENTS,
    ])

    # Wide to long
    df_dm = pd.melt(df_dm, id_vars=[
        COLUMNS.SITE_ID,
        COLUMNS.SEX,
        COLUMNS.MASKED_UPPER_BOUND_AGE_0TO2,
        COLUMNS.MASKED_UPPER_BOUND_AGE_3TO5,
        COLUMNS.MASKED_UPPER_BOUND_AGE_6TO11,
        COLUMNS.MASKED_UPPER_BOUND_AGE_12TO17,
        COLUMNS.MASKED_UPPER_BOUND_AGE_18TO25,
        COLUMNS.MASKED_UPPER_BOUND_AGE_26TO49,
        COLUMNS.MASKED_UPPER_BOUND_AGE_50TO69,
        COLUMNS.MASKED_UPPER_BOUND_AGE_70TO79,
        COLUMNS.MASKED_UPPER_BOUND_AGE_80PLUS,
    ])
    df_dm = df_dm.rename(columns={"variable": COLUMNS.AGE_GROUP, "value": COLUMNS.NUM_PATIENTS})

    # Leave only the 'upper' and 'under' values for the certain 'age_group' only
    """
    # TODO: We do not use error bars anymore
    for c in [
            COLUMNS.AGE_0TO2,
            COLUMNS.AGE_3TO5,
            COLUMNS.AGE_6TO11,
            COLUMNS.AGE_12TO17,
            COLUMNS.AGE_18TO25,
            COLUMNS.AGE_26TO49,
            COLUMNS.AGE_50TO69,
            COLUMNS.AGE_70TO79,
            COLUMNS.AGE_80PLUS
            ]:
        filter_c = df_dm[COLUMNS.AGE_GROUP] == c
        df_dm.loc[filter_c, "upper"] = df_dm.loc[filter_c, COLUMNS.NUM_PATIENTS] + df_dm.loc[filter_c, "masked_upper_bound_" + c]
        df_dm.loc[filter_c, "under"] = df_dm.loc[filter_c, COLUMNS.NUM_PATIENTS]
        df_dm.loc[filter_c, COLUMNS.NUM_PATIENTS] = df_dm.loc[filter_c, COLUMNS.NUM_PATIENTS] + df_dm.loc[filter_c, "masked_upper_bound_" + c] / 2.0
    """

    df_dm[COLUMNS.SEX] = df_dm[COLUMNS.SEX].apply(lambda x: x.capitalize())
    df_dm = df_dm[df_dm[COLUMNS.SEX] != "All"]

    # Drop unused columns
    df_dm = df_dm.drop(columns=[
        COLUMNS.MASKED_UPPER_BOUND_AGE_0TO2,
        COLUMNS.MASKED_UPPER_BOUND_AGE_3TO5,
        COLUMNS.MASKED_UPPER_BOUND_AGE_6TO11,
        COLUMNS.MASKED_UPPER_BOUND_AGE_12TO17,
        COLUMNS.MASKED_UPPER_BOUND_AGE_18TO25,
        COLUMNS.MASKED_UPPER_BOUND_AGE_26TO49,
        COLUMNS.MASKED_UPPER_BOUND_AGE_50TO69,
        COLUMNS.MASKED_UPPER_BOUND_AGE_70TO79,
        COLUMNS.MASKED_UPPER_BOUND_AGE_80PLUS,
    ])

    # Add percentage column
    unique_site_ids = df_dm[COLUMNS.SITE_ID].unique()
    for site in unique_site_ids:
        unique_sex = df_dm[df_dm[COLUMNS.SITE_ID] == site][COLUMNS.SEX].unique()
        for sex in unique_sex:
            df_filter = (df_dm[COLUMNS.SITE_ID] == site) & (df_dm[COLUMNS.SEX] == sex)
            total = df_dm.loc[
                df_filter, 
                COLUMNS.NUM_PATIENTS
            ].sum()
            
            df_dm.loc[
                df_filter, 
                "per_patients"
            ] = df_dm.loc[df_filter, COLUMNS.NUM_PATIENTS] / total * 100    
    
    return df_dm

# Process data
df_dm = read_combined_by_country_demographics_df()
df_dm = preprocess_demo_df(df_dm)

df_dm_combined = read_combined_demographics_df()
df_dm_combined = preprocess_demo_df(df_dm_combined)

# Merge
df_dm = pd.concat([df_dm, df_dm_combined])

# Use readable names
df_dm.loc[df_dm[COLUMNS.SITE_ID] == "Combined", COLUMNS.SITE_ID] = COMBINED

df_dm

# Visualizations

In [ ]:
color_scale = alt.Scale(domain=["Male", "Female"], range=COLOR20[:2])

def demographics(is_percent=False):

    demo_dropdown = alt.binding_select(options=COUNTRIES_AND_COMBINED)
    demo_selection = alt.selection_single(fields=[COLUMNS.SITE_ID], bind=demo_dropdown, name="Country", init={COLUMNS.SITE_ID: COUNTRIES_AND_COMBINED[0]})

    # Filter
    filtered_chart = alt.Chart(df_dm).transform_filter(
        demo_selection
    ).transform_filter(
        alt.datum[COLUMNS.SEX] != "Other"
    )

    DEMO_TOOLTIP = [
        alt.Tooltip(COLUMNS.SITE_ID, title="Country"),
        alt.Tooltip(COLUMNS.SEX, title="Sex"),
        alt.Tooltip(COLUMNS.AGE_GROUP, title="Age group"),
    ]

    y_field = COLUMNS.NUM_PATIENTS
    if is_percent:
        y_field = "per_patients"
        DEMO_TOOLTIP += [alt.Tooltip("per_patients", title="% of patients", format=".1f")]
    else:
        DEMO_TOOLTIP += [alt.Tooltip(COLUMNS.NUM_PATIENTS, title="# of patients")]

    # Render
    y_title = "Percentage of patients (%)" if is_percent else "Number of patients"
    bar = filtered_chart.mark_bar().encode(
        x=alt.X(f"{COLUMNS.SEX}:N", title=None, axis=None),
        y=alt.Y(f"{y_field}:Q", title=y_title, axis=alt.Axis(tickCount=5)),
        color=alt.Color(f"{COLUMNS.SEX}:N", title=None, scale=color_scale),
        tooltip=DEMO_TOOLTIP
    ).properties(width=67,height=400)

    result_vis = bar.encode(
        column=alt.Column(
            "age_group:O",
            sort=["age_0to2","age_3to5","age_6to11","age_12to17","age_18to25","age_26to49","age_50to69","age_70to79", "age_80plus"],
            header=alt.Header(labelOrient="bottom", title=None, titleOrient="bottom")
        )
    ).add_selection(
        demo_selection
    ).properties(title={
        "text": "Demographics",
        "subtitle": SUBTITLE,
        "subtitleColor": "gray",
        "anchor": "start",
        "dx": 60
    })

    return result_vis

## Demographics by country

In [ ]:
demo = apply_grouped_bar_theme(demographics(is_percent=True), strokeColor="lightgray")
demo.display()

for_website(demo, "Demographics", "Demographics by country") # TODO: Remove this before deploying notebook
# save(demo, join(SAVE_DIR, f"demographics.png".lower()), scalefactor=2.0) # Uncomment this to save *.png files